# Data Cleaning and Merging

Bernie Chen and Nahum Yared

In [ ]:
# Install for package used to filter out foreign language reviews
!pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941172 sha256=d566898eedf2c878742b2d345998832c5a48cdc1607b64cfd37bb0a62ac92c1d
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [ ]:
# Import statements
from google.colab import files
import pandas as pd
import langid
import numpy as np
import re

In [ ]:
# Import data sets for each city, replace file paths as necessary after uploading files into runtime
atl = pd.read_csv("/content/raw_atlanta_ga.csv")
boston = pd.read_csv("/content/raw_boston_ma.csv")
cary = pd.read_csv("/content/raw_cary_nc.csv")
charlotte = pd.read_csv("/content/raw_charlotte_nc.csv")
chicago = pd.read_csv("/content/raw_chicago_il.csv")
dallas = pd.read_csv("/content/raw_dallas_tx.csv")
new_york = pd.read_csv("/content/raw_newyork_ny.csv")
san_fran = pd.read_csv("/content/raw_sanfrancisco_ca.csv")
seattle = pd.read_csv("/content/raw_seattle_wa.csv")
washington = pd.read_csv("/content/raw_washington_dc.csv")
new_orleans = pd.read_csv("/content/raw_neworleans_la.csv")

In [ ]:
# prompt: remove emojis from text
def remove_emoji(string):
  emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', string)

# Define a helper function to process dataframes with review data
def helper(dataframe: pd.DataFrame, city: str) -> pd.DataFrame:
  # Add new "city" column
  dataframe["city"] = city

  # Only keep selected columns
  dataframe = dataframe[["text", "reviewId", "city"]].dropna()

  # Create a language column that encodes each review's written langauge, then drop non-english reviews
  # prompt: use langid to idenitfy the language in the text column
  dataframe["lan"] = [langid.classify(row)[0] for row in dataframe["text"]]
  dataframe = dataframe[dataframe["lan"] == "en"]

  # Remove emojis from reviews
  dataframe["text"] = dataframe["text"].apply(remove_emoji)
  dataframe = dataframe.reset_index()
  dataframe = dataframe.drop(columns=["index"])
  dataframe = dataframe.drop_duplicates("reviewId")
  return dataframe

In [ ]:
# Apply helper function to each DataFrame
atl = helper(atl, "Atlanta")
boston = helper(boston, "Boston")
cary = helper(cary, "Cary")
charlotte = helper(charlotte, "Charlotte")
chicago = helper(chicago, "Chicago")
dallas = helper(dallas, "Dallas")
new_york = helper(new_york, "New York")
san_fran = helper(san_fran, "San Francisco")
seattle = helper(seattle, "Seattle")
washington = helper(washington, "Washington")
new_orleans = helper(new_orleans, "New Orleans")

In [ ]:
# Concatenate all the DataFrames
dataframes = [atl, boston, cary, charlotte, chicago, dallas, new_york, san_fran, seattle, washington, new_orleans]
result_df = pd.concat(dataframes, ignore_index=True)
result_df

,text,reviewId,city,lan
0,I am saddened that I have to give this restaur...,ChdDSUhNMG9nS0VJQ0FnSURsaTdYY3dBRRAB,Atlanta,en
1,Come and get it!! My first visit. I ordered th...,ChZDSUhNMG9nS0VJQ0FnSURSNGNxdVV3EAE,Atlanta,en
2,Best soul food in town. If you tired of eating...,ChZDSUhNMG9nS0VJQ0FnSUNoblpPMlNBEAE,Atlanta,en
3,I would leave 0 stars if I could drove 30 mins...,ChdDSUhNMG9nS0VJQ0FnSURCMWZDYl9BRRAB,Atlanta,en
4,My first time eating here most definitely won’...,ChdDSUhNMG9nS0VJQ0FnSURCcUxLbDdRRRAB,Atlanta,en
...,...,...,...,...
7105,Peaceful! Great Service! Always delicious fo...,ChZDSUhNMG9nS0VJQ0FnSUNqLU0tcWZREAE,New Orleans,en
7106,My sister in law brought me to Landry's Restau...,ChZDSUhNMG9nS0VJQ0FnSUNqaUxLeEpnEAE,New Orleans,en
7107,We love spending an afternoon on lake Pontchar...,ChdDSUhNMG9nS0VJQ0FnSUNqZ1BIM2dnRRAB,New Orleans,en
7108,Great food & atmosphere,ChdDSUhNMG9nS0VJQ0FnSUREei1iUXN3RRAB,New Orleans,en


In [ ]:
# Export to .csv (only uncomment if this is desired)
# result_df.to_csv("all_reviews.csv", index=False, encoding="utf-8")

#Creating and Cleaning Hand-Validated Samples

In [ ]:
all_data = pd.read_csv("/content/all_reviews.csv")
all_data.head()

,text,reviewId,city,lan
0,I am saddened that I have to give this restaur...,ChdDSUhNMG9nS0VJQ0FnSURsaTdYY3dBRRAB,Atlanta,en
1,Come and get it!! My first visit. I ordered th...,ChZDSUhNMG9nS0VJQ0FnSURSNGNxdVV3EAE,Atlanta,en
2,Best soul food in town. If you tired of eating...,ChZDSUhNMG9nS0VJQ0FnSUNoblpPMlNBEAE,Atlanta,en
3,I would leave 0 stars if I could drove 30 mins...,ChdDSUhNMG9nS0VJQ0FnSURCMWZDYl9BRRAB,Atlanta,en
4,My first time eating here most definitely won’...,ChdDSUhNMG9nS0VJQ0FnSURCcUxLbDdRRRAB,Atlanta,en


In [ ]:
# Sample only 504 reviews for hand-labeling
validation_data = all_data.sample(n=504, random_state=42)
sampled_indices = validation_data.index
training_data = all_data.drop(sampled_indices)
print(f"{validation_data.count()}, {training_data.count()}, {all_data.count()}")
validation_data.head()

text        504
reviewId    504
city        504
lan         504
dtype: int64, text        6605
reviewId    6605
city        6605
lan         6605
dtype: int64, text        7109
reviewId    7109
city        7109
lan         7109
dtype: int64


,text,reviewId,city,lan
457,"Went through the drive through, bit she was ex...",ChdDSUhNMG9nS0VJQ0FnSURENDR1cW5nRRAB,Atlanta,en
1876,The food is good but the workers have really b...,ChdDSUhNMG9nS0VJQ0FnSUREOHR1Vm1nRRAB,Boston,en
6307,Great oysters and sushi - what a wonderful sur...,ChZDSUhNMG9nS0VJQ0FnSUM5aGNUNmZBEAE,New Orleans,en
3182,Very good authentic Mexican food with fast ser...,ChZDSUhNMG9nS0VJQ0FnSUM5eDdURE1BEAE,Chicago,en
1188,I recently had lunch at Love & Light and it wa...,ChZDSUhNMG9nS0VJQ0FnSUNEdlB6TUJnEAE,Boston,en


In [ ]:
# Create .csv files for our validation and training reviews (only uncomment if desired)
# validation_data.to_csv("validation_reviews.csv", index=False, encoding="utf-8")
# training_data.to_csv("training_reviews.csv", index=False, encoding="utf-8")

In [ ]:
# Download these files onto machine (only uncomment if desired)
# files.download("training_reviews.csv")
# files.download("validation_reviews.csv")

In [ ]:
# Turn hand-labeled validation data into usable format, upload to Colab Runtime and paste file path
validation_labeled = pd.read_csv("/content/validation_reviews_hand_rated.csv")
validation_labeled

,text,reviewId,city,lan,reviewer,label(s)
0,"Went through the drive through, bit she was ex...",ChdDSUhNMG9nS0VJQ0FnSURENDR1cW5nRRAB,Atlanta,en,Sunny,Empathy
1,The food is good but the workers have really b...,ChdDSUhNMG9nS0VJQ0FnSUREOHR1Vm1nRRAB,Boston,en,Sunny,Responsiveness
2,Great oysters and sushi - what a wonderful sur...,ChZDSUhNMG9nS0VJQ0FnSUM5aGNUNmZBEAE,New Orleans,en,Sunny,"Responsiveness, Reliability"
3,Very good authentic Mexican food with fast ser...,ChZDSUhNMG9nS0VJQ0FnSUM5eDdURE1BEAE,Chicago,en,Sunny,Reliability
4,I recently had lunch at Love & Light and it wa...,ChZDSUhNMG9nS0VJQ0FnSUNEdlB6TUJnEAE,Boston,en,Sunny,Reliability
...,...,...,...,...,...,...
499,Food is always fresh! The fries are never over...,ChdDSUhNMG9nS0VJQ0FnSUREdE82ZDB3RRAB,Dallas,en,Kai,Reliability
500,Food was great ribs tips A one. But the French...,ChdDSUhNMG9nS0VJQ0FnSUREaXZEMHZRRRAB,Atlanta,en,Kai,Reliability
501,I went to Morton Street Pizza and I bought a c...,ChZDSUhNMG9nS0VJQ0FnSURWLUtpTE9BEAE,Boston,en,Kai,"Reliability, Assurance"
502,The wings are cooked( too too ) hard to the po...,ChZDSUhNMG9nS0VJQ0FnSUNacy11TFFBEAE,Atlanta,en,Kai,Reliability


In [ ]:
# prompt: Given this spreadsheet, create a Python script to divide the "label(s)" column into five indicator columns, each corresponding to all possible labels present in "label(s)"
# Normalizing the labels to correct typos and inconsistent capitalization or spacing
label_corrections = {
    'reponsiveness': 'responsiveness',
    'responsivenss': 'responsiveness',
    'relaibility': 'reliability',
    'tangibility ': 'tangibility',
    'tangiblity': 'tangibility',
    'assurance': 'assurance',
    'empathy': 'empathy',
    'reliability': 'reliability',
    'responsiveness': 'responsiveness'
}

# Apply normalization and corrections
validation_labeled['normalized_labels'] = validation_labeled['label(s)'].str.lower().str.strip()\
    .replace(label_corrections, regex=True).str.split(', ')

# Extract cleaned unique labels
cleaned_unique_labels = set()
validation_labeled['normalized_labels'].dropna().apply(cleaned_unique_labels.update)

# Display cleaned and unique labels
cleaned_unique_labels

{'assurance', 'empathy', 'reliability', 'responsiveness', 'tangibility'}

In [ ]:
# Creating indicator columns for each label
for label in cleaned_unique_labels:
    validation_labeled[label] = validation_labeled['normalized_labels'].apply(lambda labels: 1 if label in labels else 0)

# Display the updated DataFrame with the new indicator columns
validation_labeled

,text,reviewId,city,lan,reviewer,label(s),normalized_labels,reliability,assurance,responsiveness,tangibility,empathy
0,"Went through the drive through, bit she was ex...",ChdDSUhNMG9nS0VJQ0FnSURENDR1cW5nRRAB,Atlanta,en,Sunny,Empathy,[empathy],0,0,0,0,1
1,The food is good but the workers have really b...,ChdDSUhNMG9nS0VJQ0FnSUREOHR1Vm1nRRAB,Boston,en,Sunny,Responsiveness,[responsiveness],0,0,1,0,0
2,Great oysters and sushi - what a wonderful sur...,ChZDSUhNMG9nS0VJQ0FnSUM5aGNUNmZBEAE,New Orleans,en,Sunny,"Responsiveness, Reliability","[responsiveness, reliability]",1,0,1,0,0
3,Very good authentic Mexican food with fast ser...,ChZDSUhNMG9nS0VJQ0FnSUM5eDdURE1BEAE,Chicago,en,Sunny,Reliability,[reliability],1,0,0,0,0
4,I recently had lunch at Love & Light and it wa...,ChZDSUhNMG9nS0VJQ0FnSUNEdlB6TUJnEAE,Boston,en,Sunny,Reliability,[reliability],1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
499,Food is always fresh! The fries are never over...,ChdDSUhNMG9nS0VJQ0FnSUREdE82ZDB3RRAB,Dallas,en,Kai,Reliability,[reliability],1,0,0,0,0
500,Food was great ribs tips A one. But the French...,ChdDSUhNMG9nS0VJQ0FnSUREaXZEMHZRRRAB,Atlanta,en,Kai,Reliability,[reliability],1,0,0,0,0
501,I went to Morton Street Pizza and I bought a c...,ChZDSUhNMG9nS0VJQ0FnSURWLUtpTE9BEAE,Boston,en,Kai,"Reliability, Assurance","[reliability, assurance]",1,1,0,0,0
502,The wings are cooked( too too ) hard to the po...,ChZDSUhNMG9nS0VJQ0FnSUNacy11TFFBEAE,Atlanta,en,Kai,Reliability,[reliability],1,0,0,0,0


In [ ]:
# Export the resulting dataframe as a .csv for easier use in analysis (only uncomment when necessary)
# validation_labeled.drop(columns=["label(s)"]).to_csv("validation_hand_labeled_cleaned.csv", index=False)